In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 

showHeader('ECMC Disclosure Dashboard',line2=f'(Drilling and) Fracking Chemicals',use_remote=True)
import itables.options as opt
opt.order = []  # no sorting

from IPython.display import HTML, display
from IPython.display import Markdown as md

# iShow(gb.reset_index(drop=True))

In [ ]:
## make ECMC version of raw_df for disclosure puposes

law_begins = datetime.datetime(year=2023,month=7,day=31)

########################
analysis_date = datetime.datetime(year=2025,month=4,day=7)
########################
deadline_days = datetime.timedelta(days=150)

archive_folder = r"C:\MyDocs\integrated\openFF_archive\raw_dataframes" # production machine
alst = os.listdir(archive_folder)
alst.sort()
print(f'Using file: {alst[-1]}')
ffdf = pd.read_parquet(os.path.join(archive_folder,alst[-1]),  # gets the most recent raw_data
                      columns=['APINumber','StateName','JobStartDate','JobEndDate','OperatorName','DisclosureId'],
                      filters = [('StateName','==','Colorado')] )
ffdf['start_date'] = pd.to_datetime(ffdf.JobStartDate,errors='coerce',format="%m/%d/%Y %I:%M:%S %p")
ffdf['end_date'] = pd.to_datetime(ffdf.JobEndDate,errors='coerce',format="%m/%d/%Y %I:%M:%S %p")
ffdf = ffdf[ffdf.end_date>law_begins]  # filter to within law's effect
ffdf.end_date = ffdf.end_date.dt.date
ffdf.start_date = ffdf.start_date.dt.date
ffdf['api10'] = ffdf.APINumber.str[:10]
FFapis = ffdf.api10.unique().tolist()

gb0 = ffdf.groupby(['DisclosureId','api10'],as_index=False)[['start_date','end_date','OperatorName']].first()
gb1 = gb0.groupby('api10',as_index=False).size().rename({'size':'num_FF_disclosures'},axis=1)
gb2 = gb0.groupby('api10',as_index=False)[['start_date','end_date','OperatorName']].first()
gbff = gb1.merge(gb2,on='api10',how='left')
# gbff[['api10','start_date','end_date','OperatorName']].sort_values('end_date')
print(f'Number of FF disclosures: {len(gbff)}')
print(f'Earliest start date: {gbff.start_date.min()}')
print(f'Latest end date: {gbff.end_date.max()}')

In [ ]:
## get new well data set
import scrape.Colorado.CO_data_helper as cdh
wells_df = cdh.fetch_full_wells_df()

# sort by spud, so most recent spud is kept if there are duplicates
wells_df = wells_df.sort_values('spud')
# drop well_df duplicates
wells_df = wells_df[~wells_df.api10.duplicated(keep='last')]

In [ ]:
## get locationID into FF data

mg = pd.merge(gbff,wells_df[['api10','location','Operator','spud']],on='api10',how='left')
mg = mg.sort_values('end_date')
ffsum = mg.groupby('location',as_index=False)[['spud','Operator','start_date','end_date','OperatorName']].last()
gb1 = mg.groupby('location',as_index=False).size().rename({'size':'num_FF_disclosures'},axis=1)
mg = ffsum.merge(gb1,on='location',how='left')

# mg

In [ ]:
# get current ECMC disclosures and add to table
url = "https://ecmc.state.co.us/depot/Stats/ChemicalDisclosures/Index/Locations"
loc_nb_dir = r"G:\My Drive\Info_home\Projects\Project_Homes\ECMC_new_disclosure_instrument\location_webpages"

tables = pd.read_html(url)
t = tables[0]
t.columns = ['location','loc_name','ECMC_last_date']
emg = mg.merge(t,on='location',how='outer')
emg.num_FF_disclosures = emg.num_FF_disclosures.fillna('no_FF')
emg.loc_name = emg.loc_name.fillna('no_ECMC')
emg.ECMC_last_date = emg.ECMC_last_date.fillna(' -- ')

emg['nblink'] = th.wrap_URL_in_html(
locs_with_disc = emg.location.tolist()
display(md(f'#### Num locations with no ECMC disclosures: {len(emg[emg.loc_name=="no_ECMC"])}'))
display(md(f'#### Num locations with no FF disclosures: {len(emg[emg.num_FF_disclosures=="no_FF"])}'))
emg

In [ ]:
# make wells df for recent spuds, not in other table already

display(md("### Summary for wells with no FF or ECMC disclosure"))
c = wells_df.spud>=law_begins
gb1 = wells_df[c].groupby('location',as_index=False)[['spud']].max()
gb2 = wells_df[c].groupby('location',as_index=False)[['Operator']].first()
gb3 = wells_df[c].groupby('location',as_index=False).size().rename({'size':'num_ECMC_wells'},axis=1)

gbwells = pd.merge(gb1,gb2,on='location',how='left')
gbwells = gbwells.merge(gb3,on='location',how='left')
not_disc =~gbwells.location.isin(locs_with_disc) 

display(md(f"#### Total number of wells with spud date but no disclosure: {gbwells.num_ECMC_wells.sum():,}"))
gbwells[not_disc]

In [ ]:
display(md("### Well Operators summary for no FF disclosure"))
display(md("For **locations** with no disclosure from either instrument"))
gbwells[not_disc].groupby('Operator',as_index=False)['num_ECMC_wells'].sum()

# Older stuff

In [ ]:
# ## Fetch the well database directly from the colorado web page - Gemini generated
# ## takes about 30 seconds

# import requests
# import zipfile
# import io
# from dbfread import DBF
# import pandas as pd
# import tempfile
# import os

# def read_wells_dbf_from_zip(url):
#     """
#     Downloads a zip file, extracts Wells.dbf, and reads it into a pandas DataFrame.

#     Args:
#         url (str): The URL of the zip file.

#     Returns:
#         pandas.DataFrame: A DataFrame containing the data from Wells.dbf, or None if an error occurs.
#     """
#     try:
#         response = requests.get(url)
#         response.raise_for_status()

#         with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
#             dbf_data = zip_file.read("Wells.dbf")

#         # Create a temporary file to store the DBF data
#         with tempfile.NamedTemporaryFile(delete=False) as temp_file:
#             temp_file.write(dbf_data)
#             temp_file_path = temp_file.name

#         # Read the DBF file from the temporary file path
#         dbf = DBF(temp_file_path)
#         df = pd.DataFrame(iter(dbf))

#         # Delete the temporary file
#         os.unlink(temp_file_path)

#         return df

#     except requests.exceptions.RequestException as e:
#         print(f"Error downloading the file: {e}")
#         return None
#     except zipfile.BadZipFile as e:
#         print(f"Error with the zip file: {e}")
#         return None
#     except KeyError as e:
#         print(f"Error: 'Wells.dbf' not found in the zip archive. {e}")
#         return None
#     except Exception as e:
#         print(f"An unexpected error occurred: {e}")
#         return None

# # URL of the zip file
# url = "https://ecmc.state.co.us/documents/data/downloads/gis/WELLS_SHP.ZIP"

# # Read the DBF file into a DataFrame
# wells_df = read_wells_dbf_from_zip(url)
# # print(wells_df.columns)
# if wells_df is not None:
#     wells_df['api10'] = wells_df.API_Label.str.replace('-','')
#     wells_df['spud'] = pd.to_datetime(wells_df.Spud_Date)
#     wells_df['location'] = wells_df.Loc_ID
#     wells_df = wells_df[['api10','Operator','spud','location','Facil_Stat']]
# else:
#     print("Failed to read the Wells.dbf file.")

# wells_df = wells_df.sort_values('spud')
# wellsgb = wells_df[wells_df.location.notna()].groupby('location',as_index=False)['spud'].last()
# gb1 = wells_df[wells_df.location.notna()].groupby('location',as_index=False).size().rename({'size':'num_wells_recs'},axis=1)
# wellsgb = wellsgb.merge(gb1,on='location',how='left')

In [ ]:
# wells_df

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import datetime

# def plot_cumulative_spuds(df, date_column='spud', start_date='2023-08-01'):
#     """
#     Plots the cumulative number of wells spudded over time from a given start date.

#     Args:
#         df (pd.DataFrame): DataFrame containing spud date information.
#         date_column (str): Name of the column containing the spud dates.
#         start_date (str): Start date for cumulative calculation (YYYY-MM-DD).
#     """

#     # Convert start_date string to datetime object
#     start_date_dt = pd.to_datetime(start_date)

#     # Ensure the date column is of datetime type
#     df[date_column] = pd.to_datetime(df[date_column])

#     # Filter for dates after the start date and today
#     c1 = df[date_column] >= start_date_dt
#     c2 = df[date_column] < datetime.datetime.today()
#     df = df[c1&c2].copy()  # Use .copy() to avoid SettingWithCopyWarning

#     # Keep only producing or waiting for completion wells (shut in, abandonded, etc are ignored)
#     # c3 = df.Facil_Stat.isin(['PR','WO'])
#     # df_filtered = df[c3].copy()
#     df_filtered = df.copy()
#     # print(f'Wells spudded but excluded by current status: {len(df)-len(df_filtered)}')
#     vc = df_filtered.Facil_Stat.value_counts().reset_index()
#     status_descriptions = {
#         'PR': 'Producing',
#         'DG': 'Drilling',
#         'PA': 'Plugged and Abandoned',
#         'TA': 'Temporarily Abandoned',
#         'SI': 'Shut-In',
#         'WO': 'Waiting on Completion',
#         'AP': 'Active Permit',
#         'IJ' : 'Injecting',
#         'SO' : 'Suspended Operation'
#         # Add other status codes and their descriptions here
#     }
#     desc = []
#     for i,row in vc.iterrows():
#         desc.append(status_descriptions[row.Facil_Stat])
#     vc["Description"] = desc
#      # Print the table
#     display(md('# Disclosure of Drilling Chemicals'))
#     # display(md("## Wells with spud date within law's effect"))
#     display(md('### Current number of wells with discloure of drilling chemicals: 0'))
    
#     if df_filtered.empty:
#         print("No spud dates found after the specified start date.")
#         return

#     # Daily cumulative plot
#     daily_counts = df_filtered.groupby(df_filtered[date_column].dt.date).size().cumsum()
    
#     plt.figure(figsize=(6,6))
#     plt.plot(daily_counts.index, daily_counts.values, marker='o', linestyle='-')
#     plt.title('Cumulative number of wells with spud date \nsince law went into effect')
#     plt.xlabel('Spud Date')
#     plt.ylabel('Cumulative Number of Wells')
#     plt.grid(True)
#     plt.xticks(rotation=45)
 
#     # Annotations
#     first_day_dt = pd.to_datetime('2023-08-01').date()
#     deadline_dt = (pd.to_datetime('2023-08-01') + pd.Timedelta(days=150)).date()

#     # Find the y-values for the annotations
#     first_day_y = daily_counts.get(first_day_dt, 0)  # Get y-value, default to 0 if not found
#     deadline_y = 0 #daily_counts.get(deadline_dt, 0)

#     # Annotate the first day
#     plt.annotate(
#         'July 31, 2023 - Law goes into effect',
#         xy=(first_day_dt, first_day_y),
#         xytext=(first_day_dt, first_day_y + 700),  # Adjust y-offset as needed
#         arrowprops=dict(facecolor='orange', shrink=0.05),
#         horizontalalignment='left'
#     )

#     # Annotate the deadline
#     plt.annotate(
#         'Deadline for first disclosures\n (150 days after start of downhole operation)',
#         xy=(deadline_dt, 0),
#         xytext=(deadline_dt, deadline_y + 50),  # Adjust y-offset as needed
#         arrowprops=dict(facecolor='red', shrink=0.05),
#         horizontalalignment='left'
#     )

   
#     plt.tight_layout()
#     plt.show()

#     return vc, df_filtered

# vc, df_filtered = plot_cumulative_spuds(wells_df)

In [ ]:
# display(md("\n\n### Status codes for wells drilled within law's effect"))
# iShow(vc.reset_index(drop=True))

In [ ]:
# display(md("\n### Operators for wells drilled within law's effect"))
# iShow(df_filtered.Operator.value_counts())

In [ ]:
# ### get most recent FF - raw data
# import datetime
# law_begins = datetime.datetime(year=2023,month=7,day=31)

# archive_folder = r"C:\MyDocs\integrated\openFF_archive\raw_dataframes" # production machine
# alst = os.listdir(archive_folder)
# alst.sort()
# ffdf = pd.read_parquet(os.path.join(archive_folder,alst[-1]),  # gets the most recent raw_data
#                       columns=['APINumber','StateName','JobStartDate','JobEndDate','OperatorName','DisclosureId'],
#                       filters = [('StateName','==','Colorado')] )
# ffdf['start_date'] = pd.to_datetime(ffdf.JobStartDate,errors='coerce',format="%m/%d/%Y %I:%M:%S %p")
# ffdf['end_date'] = pd.to_datetime(ffdf.JobEndDate,errors='coerce',format="%m/%d/%Y %I:%M:%S %p")
# ffdf = ffdf[ffdf.end_date>law_begins]  # filter to within law's effect
# ffdf.end_date = ffdf.end_date.dt.date
# ffdf['api10'] = ffdf.APINumber.str[:10]
# FFapis = ffdf.api10.unique().tolist()

# gbff = ffdf.groupby('DisclosureId',as_index=False)[['api10','start_date','end_date','OperatorName']].first()
# gb1 = ffdf.groupby('DisclosureId',as_index=False).size().rename({'size':'num_FF_disclosures'},axis=1)
# # gbff = gbff.merge(gb1,on='DisclosureId',how='left')
# # gbff[['api10','start_date','end_date','OperatorName']].sort_values('end_date')
# gbff

In [ ]:
# ## get locationID into FF data

# mg = pd.merge(gbff,wells_df[['api10','location','spud']],on='api10',how='left')
# mg = mg.sort_values('end_date')
# ffsum = mg.groupby('location',as_index=False)[['start_date','end_date','OperatorName','spud']].last()
# gb1 = mg.groupby('location',as_index=False).size().rename({'size':'num_FF_disclosures'},axis=1)
# mg = ffsum.merge(gb1,on='location',how='left')
# mg

In [ ]:
# ## get the ECMC disclosure locations table.
# url = "https://ecmc.state.co.us/depot/Stats/ChemicalDisclosures/Index/Locations"
# t = pd.read_html(url)
# # print(len(t))
# eLocs = t[0].copy()
# eLocs['location'] = eLocs['Location Number']
# eLocs['last_update'] = pd.to_datetime(eLocs['Last Disclosure Update'],errors='coerce',format="%m/%d/%Y %I:%M:%S %p")
# eLocs

In [ ]:
# ## Now fetch each location webpage and the imaged_doc names
# # rooturl = 'https://ecmc.state.co.us/depot/Stats/ChemicalDisclosures/Location/'
# import scrape.Colorado.CO_imaged_docs_extract as cide
# import re

# pdf_dir = r"G:\My Drive\webshare\scrape_data\Colorado\imaged_docs"

# outdf = pd.DataFrame()
# for i,row in eLocs.iterrows():
#     apis = set()

#     out = pd.read_html(f"https://ecmc.state.co.us/depot/Stats/ChemicalDisclosures/Location/{row.location}")
#     file_tab = out[1]
#     for j,jrow in file_tab.iterrows():
#         docnum = jrow['Document #']
#         pdffn = os.path.join(pdf_dir,f'{docnum}.pdf')
#         try:
#             text = cide.get_pdf_text(pdffn)
#             fapis = cide.extract_APINumbers(text)
#             for api in fapis:
#                 # some apis are misformed. Strip all non-numeric.
#                 s = re.sub('[^\d]','', api)
#                 apis.add(s)
#         except:
#             print(f'Couldnt fetch from location {row.location}, file: {docnum}')
#     tmp = pd.DataFrame({'api10':list(apis),'location':row.location,'date':row.last_update})
#     outdf = pd.concat([outdf,tmp])
# # outdf

In [ ]:
# # lets plot

# # FF Daily cumulative plot
# FF_daily_counts = gbff.groupby(gbff.start_date.dt.date).size().cumsum().reset_index()
# FF_daily_counts.columns=['date','FF_counts']

# #expected
# expected = FF_daily_counts.copy()
# expected.date = expected.date + datetime.timedelta(days=150) 
# # cutoff_date_dt = pd.to_datetime(datetime.datetime(year=2023,month=7,day=31)).date()


# # ECMC Daily cumulative plot
# ECMC_daily_counts = outdf.groupby(outdf.date.dt.date).size().cumsum().reset_index()
# ECMC_daily_counts.columns=['date','ECMC_counts']

# spacer = datetime.timedelta(days=20)

# plt.figure(figsize=(6, 6))
# plt.plot(FF_daily_counts.date, FF_daily_counts.FF_counts, linestyle='-',color='grey')# label=label1)
# plt.plot(expected.date, expected.FF_counts, marker='o', linestyle='-',)# label=label1)
# plt.plot(ECMC_daily_counts.date, ECMC_daily_counts.ECMC_counts, marker='o', linestyle='-',)# label=label1)
# # plt.plot(df2[date_column], df2[counts_column], marker='o', linestyle='-', label=label2)

# plt.title('ECMC chemical disclosures by number of wells\nin reference to FracFocus disclosures')
# plt.xlabel('Date')
# plt.ylabel('Counts')
# plt.xlim(FF_daily_counts.date.min()-spacer,ECMC_daily_counts.date.max()+spacer)
# plt.grid(True)
# plt.xticks(rotation=45)
# plt.legend(['fracking start date (by FF data)',
#             'expected disclosure',
#             'Actual disclosure to ECMC'])  # Add legend to distinguish the lines


# # Annotations
# first_day_dt = pd.to_datetime('2023-08-01').date()
# deadline_dt = (pd.to_datetime('2023-08-01') + pd.Timedelta(days=150)).date()

# # Find the y-values for the annotations
# first_day_y = FF_daily_counts.get(first_day_dt, 0)  # Get y-value, default to 0 if not found
# deadline_y = 0 #daily_counts.get(deadline_dt, 0)

# # Annotate the first day
# plt.annotate(
#     'July 31, 2023 \nLaw goes into effect',
#     xy=(first_day_dt, first_day_y),
#     xytext=(first_day_dt, first_day_y + 900),  # Adjust y-offset as needed
#     arrowprops=dict(facecolor='orange', shrink=0.05),
#     horizontalalignment='left'
# )

# # Annotate the deadline
# plt.annotate(
#     'First disclosures\nexpected',
#     xy=(deadline_dt, 0),
#     xytext=(deadline_dt, deadline_y + 700),  # Adjust y-offset as needed
#     arrowprops=dict(facecolor='red', shrink=0.05),
#     horizontalalignment='center'
# )

# display(md('---\n# Diclosure of fracking chemical disclosures '))
# display(md('### Using FracFocus disclosures as a reference\n\n'))

# plt.tight_layout()
# plt.show()



In [ ]:
# ## add in wells_df locations that are within the law, but not in FF or ECMC
# c = wells_df.spud>law_begins
# c1 = wells_df.api10.isin(FFapis)
# recent_wells = wells_df[c& ~(c1)]
# gb1 = recent_wells.groupby('location',as_index=False).size()
# gb2 = recent_wells.groupby('location',as_index=False)[['Operator']].last()
# wells_extra = pd.merge(gb1,gb2,on='location',how='left')
# extra_locs = wells_extra.location.unique().tolist()
# wells_extra

In [ ]:
# # are there locations in extras that aren't in FF
# mg[mg.location.isin(extra_locs)]

In [ ]:
# print(wells_df.columns)

In [ ]:
# last_repo = datetime.datetime(year=2025,month=3,day=24)
# law_begins = datetime.datetime(year=2023,month=7,day=31)

# # fetch data set
# df = pd.read_parquet(os.path.join(hndl.sandbox_dir,'workdf.parquet'),
#                columns=['APINumber','DisclosureId','bgStateName','date','OperatorName','bgCAS','mass',
#                        'bgCountyName','in_std_filtered','JobStartDate','TradeName'],
#                filters=[('bgStateName','==','colorado'),
#                         ('in_std_filtered','==',True),
#                         ]
#               )

# df = df[df.date>law_begins]
# df['start_date'] = pd.to_datetime(df.JobStartDate,errors='coerce').dt.date
# df['start_date'] = pd.to_datetime(df.start_date,errors='coerce')
# gb = df.groupby('DisclosureId',as_index=False)[['APINumber','OperatorName','bgCountyName',
#                                                 'start_date','date']].first()
# # gb1 = gb.groupby('bgStateName',as_index=False)['DisclosureId'].count().rename({'DisclosureId':'Number of disclosures'},axis=1)

# # gb2 = gb.groupby(['bgStateName','bgCountyName'],as_index=False)['DisclosureId'].count()
# # gb2 = gb2.groupby('bgStateName',as_index=False)['bgCountyName'].count().rename({'bgCountyName':'Number of counties'},axis=1)

# # gb3 = gb.groupby('bgStateName',as_index=False)['date'].agg(['min','max']).rename({'min':'earliest',
# #                                                                                   'max':'latest'},axis=1)
# # mg = pd.merge(gb1,gb2,on='bgStateName',how='left')
# # mg = pd.merge(mg,gb3,on='bgStateName',how='left')
# # mg = mg.rename({'bgStateName':'State'},axis=1)
# # # mg = mg.reset_index(drop=True)
# # mg.State = '<center><h3>'+mg.State.str.title().map(lambda x: th.getStateLink(x,x))+'</h3></center>'
# # mg = mg.sort_values('Number of disclosures',ascending=False)

# import itables.options as opt
# opt.order = []  # no sorting

# iShow(gb.reset_index(drop=True))
# # gb.info()

# work_dir = r"G:\My Drive\webshare\scrape_data\Colorado\ECMC_tracker"
# welldf_fn = os.path.join(work_dir,'ECMC_welldf.parquet')

# # First time only
# # welldf = pd.DataFrame({'APINumber':[],'DisclosureId':[],
# #                        'locationID':[],'OperatorName':[],
# #                        'ECMC_op':[],'bgCountyName':[],
# #                        'start_date':[],'date':[]})
# # welldf.to_parquet(welldf_fn)
# welldf = pd.read_parquet(welldf_fn)
# welldf

### Populate welldf with current disclosures

In [ ]:
dlist = welldf.DisclosureId.unique().tolist()
c = ~gb.DisclosureId.isin(dlist)
both = pd.concat([welldf,gb[c]])
both

### Fetch COGIS data on all disclosures

In [ ]:
####  REDO WHEN ADDING NEW DATA

# import scrape.Colorado.CO_scraper_support as coss
# def make_soup_for_well(APINumber):
#     rooturl = "https://ecmc.state.co.us/cogisdb/Facility/FacilityDetailExpand?api="
#     url = rooturl+ APINumber[2:-4]
#     # print(url)
#     html_content = coss.fetch_html(url)
#     # print(html_content)
#     # print('\n'+url)
#     # print("Recycled water" in html_content)
#     return coss.make_soup(html_content)

# def get_location_id(soup):
#     # Find the <td> element containing "LocationID:"
#     location_id_td = soup.find('td', string='LocationID:')
    
#     if location_id_td:
#         # Get the next sibling (the <td> with the value)
#         value_td = location_id_td.find_next_sibling('td')
#         if value_td:
#             # Extract the text and convert it to an integer
#             location_id_value = int(value_td.text)
#             return location_id_value  # Output: 455576
#         else:
#             print("Error: Could not find sibling td with the value.")
#     else:
#         print("Error: Could not find td containing 'LocationID:'") 
    

# def get_ECMC_operator(soup):
#     # Find the <td> element containing "Operator:"
#     operator_td = soup.find('td', string='Operator:')
    
#     if operator_td:
#         # Get the next sibling (the <td> with the value)
#         value_td = operator_td.find_next_sibling('td')
#         if value_td:
#             # Extract the text
#             operator_value = value_td.text.strip()  # Remove leading/trailing spaces
#             output = operator_value.split('(')[0]
#             return output
#         else:
#             print("Error: Could not find sibling td with the value.")
#     else:
#         print("Error: Could not find td containing 'Operator:'")    

# #  fetch ECMC data for the new disclosures
# def get_ECMC_data(APINumber):
#     soup = make_soup_for_well(APINumber)
#     locID = get_location_id(soup)
#     ECMCop = get_ECMC_operator(soup)
#     return (locID,ECMCop)
# locs = []
# ops = []

# # tmp = both[:5].copy()
# for i,row in both.iterrows():
#     print(i,end=' ')
#     if type(row.locationID) == 'str':
#         print(f'skipping {row.APINumber}')
#         locs.append(row.locationID)
#         ops.append(row.ECMC_op)
#         continue
#     else:
#         tup = get_ECMC_data(row.APINumber)
#         locs.append(tup[0])
#         ops.append(tup[1])
# # 
# both.locationID = locs
# both.ECMC_op = ops
# both.to_parquet(welldf_fn)
# both

## get ECMC locations

In [ ]:
# url = "https://ecmc.state.co.us/depot/Stats/ChemicalDisclosures/Index/Locations"
# t = pd.read_html(url)
# # print(len(t))
# eLocs = t[0].copy()
# eLocs['locationID'] = eLocs['Location Number']
# mg = pd.merge(both,eLocs[['locationID','Location Name','Last Disclosure Update']],on='locationID', how='outer')
# print(f'Total number of unique "locations" (well pads): {len(mg.locationID.unique())}')
# c = mg['Location Name'].notna()
# print(f'Number of locations with an ECMC disclosure: {len(mg[c].locationID.unique())}')
# mg['status'] = np.where(mg.APINumber.isna(),'no_FF','has_FF')
# mg.status = np.where(mg['Location Name'].isna(), mg.status + ' no_ECMC', mg.status + ' has_ECMC')
# mg

### Show data from one location

In [ ]:
# target_loc = 477204
# ff_api = mg[mg.locationID==target_loc].APINumber.unique().tolist()
# print(f'Number of wells in FF at this location: {len(ff_api)}')
# print(ff_api)
# ff_cas = df[df.APINumber.isin(ff_api)].groupby('bgCAS', as_index=False)['mass'].sum()
# ff_cas
# ff_tn = df[df.APINumber.isin(ff_api)].groupby('TradeName', as_index=False).size()
# ff_tn

In [ ]:
# out = pd.read_html(f"https://ecmc.state.co.us/depot/Stats/ChemicalDisclosures/Location/{target_loc}")
# ecmc_cas = out[0]
# casmg = pd.merge(ecmc_cas,ff_cas,left_on='CAS Number', right_on='bgCAS',how='outer')
# casmg

In [ ]:
# ecmc_form46a = out[1]
# ecmc_form46a['date'] = pd.to_datetime(ecmc_form46a['Submit Date'])
# c = ecmc_form46a.date==ecmc_form46a.date.max()
# pdf_dir = r"G:\My Drive\webshare\scrape_data\Colorado\imaged_docs"
# docnum = ecmc_form46a[c]['Document #'].tolist()[0]             
# pdffn = os.path.join(pdf_dir,f'{docnum}.pdf')
# pdffn

In [ ]:
# known_ecmc_prod_names = pd.read_parquet(os.path.join(pdf_dir,'products_set.parquet'))
# known_ecmc_prod_names

In [ ]:
# import scrape.Colorado.CO_imaged_docs_extract as cide
# text = cide.get_pdf_text(pdffn)
# ecmc_prod_names = cide.extract_product_names(text,known_ecmc_prod_names.TradeName.unique().tolist())
# ecmc_api10 = cide.extract_APINumbers(text)
# print(ecmc_prod_names)

In [ ]:
# prod_names = pd.DataFrame({'TradeName':df.TradeName.unique().tolist()})
# prod_names.to_parquet(os.path.join(r"C:\MyDocs\integrated\gwa_local\tmp",'CO_prod_names.parquet'))

## get well data for spud date

In [ ]:
# fn = r"G:\My Drive\webshare\scrape_data\Colorado\Wells.csv"
# cowells = pd.read_csv(fn,low_memory=False,usecols=['API,C,10','Spud_Date,D','Operator,C,50'],
#                      dtype={'API,C,10':'str'})
# cowells.columns = ['api','operator','spud']
# cowells['spud_date'] = pd.to_datetime(cowells.spud)
# cowells['APINumber'] = '05'+cowells.api+'0000'
# cowells = cowells.drop(['api','spud'],axis=1)
# cowells

In [ ]:
mg = mg.merge(cowells[['spud_date','APINumber']],on='APINumber',how='left')
mg['spud_start_diff_days'] = (mg.start_date - mg.spud_date).dt.days
mg